In [ ]:
%%html
<style>
h1 { color: rgb(226, 0, 26); }
h2 { color: rgb(0, 80, 120); }
</style>

# BigData - DBWeather
---
Programmentwurf - Wintersemester 2022 - DHBW Mannheim\
Daniel Czeschner, Kai Grübener, Frederik Wolter

## 0. Inhalt

1. Business Understanding

2. Data Understanding

3. Data Preperation

4. Modeling

5. Evaluation

6. Deployment

## 1. Business Understanding //Kai


//Motivation aus Anwendungssicht: welche Fragestellungen werden mit Hilfe der Daten beantwortet?

* Deutsche Bahn - Analyse

* Ziel: Datenerhebung & -auswertung zu Zugverspätungen / -ausfällen
    * Grundlegende statistische Darstellung von Fahrplandaten
    * Identifizieren von Problemen und Ableiten möglicher Lösungswege
    * Zusätzlich Erhebung von Wetterdaten & Korrelationsanalyse
        * Vermutung von Mechanikern: Klimaanlagen von ICE's sorgen für Störungen im Betrieb

## 2. Data Understanding //Kai

//Beschreibung der Datenquellen
//Welche Schwierigkeiten sind aufgetreten und wie wurden sie gelöst? (bzgl. Datenquelle der Deutschen Bahn evtl.)

* Verwendete Datenquellen:
    * DB Reiseauskunft API
    * BrightSky API

## 2. Data Understanding

### DB Reiseauskunft API

* (Inoffizielle) API der Deutschen Bahn
* Stellt standortbezogen u.A. folgende Daten bereit:
    * Zeitstempel
    * Zugtyp
    * Zugnummer
    * Abfahrt / Ankunft
    * Verspätung / Zugausfall
    * ...
* //Andere API hier erwähnen?

## 2. Data Understanding

### BrightSky API

* API des Deutschen Wetterdienstes
* Stellt standortbezogen u.A. folgende Wetterdaten bereit:
    * Zeitstempel
    * Temperatur
    * Wetterkonditionen
    * Windstärke / -richtung
    * Luftfeuchtigkeit
    * Anzahl Sonnenstunden
    * Bewölkung
    * ...

## 3. Data Preparation //Kai

//Auswahl, Formatierung, Transformation der Daten

* DB Fahrplan API
    * Abfrage alle 30 Minuten
    * Überprüfung der Vollständigkeit
    * Parsen der zurückgegebenen Daten
    * Abspeichern der Daten in MongoDB (JSON-Format)
* BrightSky Wetter API
    * Abfrage alle 30 Minuten
    * Überprüfung der Vollständigkeit
    * Abspeichern der zurückgegebenen Daten (JSON-Format) in MongoDB
    
//Grafik einfügen oder so belassen??


## 4. Modeling

* Zwei Python-Skripte zum abfragen der Daten von den APIs.
* Daten werden in das JSON-Format geparst und in einer MongoDB (NoSQL-Datenbank) gespeichert.
* Die Skripte werden durch den Server der Datenbank via cronjobs ausgeführt.
 * Immer zu jeder halben und vollen Stunde
* Die Skripte loggen ihre Ausführung in eine Log-Datei auf den Server die wie folgt aussieht:
<center>
$
2022-11-03 19:00:01,408 INFO    weather: Start main_weather execution ...
2022-11-03 19:00:01,410 INFO    train: Start main_train execution ...
2022-11-03 19:00:01,708 INFO    weather: Result lat=50.05 lon=8.6 has 25 elements
2022-11-03 19:00:01,824 INFO    weather: Result lat=49.5 lon=8.48 has 25 elements
2022-11-03 19:00:01,844 INFO    weather: finished: 0 inserted, 2 updated, 48 unchanged
2022-11-03 19:00:01,844 INFO    weather: ###########################################
2022-11-03 19:00:01,917 INFO    train: Result eva=8000105 type=arr has 690 lines, calc 230.0 sets
2022-11-03 19:00:02,493 INFO    train: Result eva=8000105 type=dep has 672 lines, calc 224.0 sets
2022-11-03 19:00:02,938 INFO    train: Result eva=8000244 type=arr has 291 lines, calc 97.0 sets
2022-11-03 19:00:03,233 INFO    train: Result eva=8000244 type=dep has 264 lines, calc 88.0 sets
2022-11-03 19:00:03,301 INFO    train: finished: 295 inserted, 119 updated, 225 unchanged
2022-11-03 19:00:03,301 INFO    train: ###########################################
$
</center>

## 4. Modeling

* Die MongoDB wurde dabei so konfiguriert, dass eine authentifizierung der Clients notwendig ist.
 * Dadurch kann global auf die Datenbank zugegriffen werden
 * Zudem wurde ein Web-Dashboard aufgesetzt, worüber die gepspeicherten Daten einfach eingesehen werden können:
<img src="img/MongoDBDashboard.png" width="1027" height="346"/>

## 4. Modeling

* Die gesamte Pipeline sieht nun wie folgt aus:

<center><img src="img/DataPipeline.png" /></center>

## 5. Evaluation

FREDERIK
* Hauptteil: Beschreibung und Erläuterung der erstellten Datenpipeline
* Wurde eine spezielle Konfiguration verwendet?
* Darstellung der Ergebnisse mit einfacher Visualisierung (z.B. Jupyter oder Streamlit)

## 6. Deployment

?
* Zusammenfassung: Beantwortung der Fragen aus Anwendungssicht
  * noch nicht genut Daten für zuverlässige Aussage
* (was hätte man anders machen können?)

# Temp Test 

In [ ]:
import pandas as pd
from database import Database

import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 200
mpl.rcParams["figure.autolayout"] = True
mpl.rcParams["figure.figsize"] = [10.00, 4.00]

database = Database()

## Wetter Test 

In [ ]:
agg_result = database.mongo_data_weather.aggregate([{
    "$group": {
        "_id": "$timestamp",
        "count": {"$sum": 1}
    }
}])
weather = pd.DataFrame(list(agg_result))
#weather
plot = weather.plot(x='_id', y='count')

In [ ]:
agg_result3 = database.mongo_data_weather.aggregate([
    {
        "$group": {
            "_id": {
                "$dateToString": {
                    "format": "%d.%m.%Y",
                    "date": {"$toDate": "$timestamp"}
                }
            },
            "count": {"$sum": 1}
        }
    },
    {"$sort": {"_id": 1}}
])
weather2 = pd.DataFrame(list(agg_result3))
#weather2
weather2.plot(x='_id', y='count')

In [ ]:
weather2

In [ ]:
result = database.mongo_data_weather.find({})
df = pd.DataFrame(list(result))
df

In [ ]:
df.plot(x='timestamp', y='temperature')
#weather
print(list(result))

## Züge

In [ ]:
database.mongo_data_train.count_documents(filter={})

In [ ]:
for i in database.mongo_data_train.find({"eva": 8000244}).limit(5):
    print(i)

In [ ]:
agg_result2 = database.mongo_data_train.aggregate([{
    "$group": {
        "_id": "$timestamp",
        "count": {"$sum": 1}
    }
}])
#for i in agg_result:
#    print(i)
train = pd.DataFrame(list(agg_result2))
#train
train.plot(x='_id', y='count')

In [ ]:
agg_result4 = database.mongo_data_train.aggregate([
    {
        "$group": {
            "_id": {
                "$dateToString": {
                    "format": "%d.%m.%Y",
                    "date": {"$toDate": "$timestamp"}
                }
            },
            "count": {"$sum": 1}
        }
    },
    {"$sort": {"_id": 1}}
])
#for i in agg_result:
#    print(i)
train2 = pd.DataFrame(list(agg_result4))
#train2
train2.plot(x='_id', y='count')

In [ ]:
train2

In [ ]:
for i in database.mongo_data_train.find(sort=[("delay", -1)]).limit(20):
    print(i)

In [ ]:
for i in database.mongo_data_train.find(filter={"con_type":"RE"}, sort=[("delay", -1)]).limit(20):
    print(i)

In [ ]:
timerange = database.mongo_data_train.find({
    "timestamp": {
        "$gt": "2022-11-05T17:45",
        "$lt": "2022-11-05T18:00",
    }
})
l = list(timerange)
for i in l:
    print(i)
print(len(l))

In [ ]:
database.close()

In [ ]:
# use https://stackoverflow.com/a/33079582/13777031

# https://rise.readthedocs.io/en/stable/customize.html

# https://ourcodingclub.github.io/tutorials/pandas-time-series/